In [62]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

# Definition des fonctions

In [63]:
def neural_networks():

    model = models.Sequential()

    model.add(layers.Dense(128, activation='relu', input_shape=(100*19,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(24, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


# Construction base de données

In [64]:
df=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
df_result=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv")   

In [65]:
df['bid_ask_spread']=df['ask']-df['bid']
df['trade']=df['trade'].astype(int)

In [66]:
value_categor=['venue','action','side']

for k in value_categor:

    df_encoded = pd.get_dummies(df[k], prefix=k).astype(int)
    df = df.merge( df_encoded , left_index = True , right_index = True)
    df.drop( columns=[k] , inplace = True )

df.drop(columns=['obs_id','order_id'],inplace=True)


In [67]:
value_no_center=['price' , 'bid' , 'ask' , 'bid_size', 'ask_size', 'bid_ask_spread' ]

for k in value_no_center:
    
    df[k] = df[k] / max( df[k] )


In [68]:
X=df.values.reshape(-1, 19 * 100)

# Construction modèle

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, df_result['eqt_code_cat'].array, test_size=0.2, random_state=42)

In [70]:
model_1 = neural_networks()
model_1.fit(X_train, y_train, epochs=10, batch_size=64)

predictions_prod = model_1.predict(X_test)
y_pred=[]
for liste in predictions_prod:
    top = liste.argmax()
    y_pred += [top]
    
accuracy_score(y_pred,y_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0464 - loss: 5.0032
Epoch 2/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0789 - loss: 3.0621
Epoch 3/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0969 - loss: 2.9196
Epoch 4/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1048 - loss: 2.8428
Epoch 5/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1138 - loss: 2.7999
Epoch 6/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1187 - loss: 2.7673
Epoch 7/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1172 - loss: 2.7483
Epoch 8/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1227 - loss: 2.7229
Epoch 9/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1242 - loss: 2.7132
Epoch 10/10
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1259 - loss: 2.6985
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step


0.11318407960199005